In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [ ]:
def arm(database, min_sup, min_con):
    """
    Function:Association rule mining
    Input:Minimum Support, Minimum Confidence
    Output:association rules
    """
    tran = []
    for i in range(database.shape[0]):
        tran.append([str(database.values[i, j]) for j in range(database.shape[1])])

    tran = np.array(tran)
    te = TransactionEncoder()
    tran_te = te.fit(tran).transform(tran)
    tran_df = pd.DataFrame(tran_te, columns=te.columns_)

    frequent_items = apriori(tran_df, min_support=min_sup, use_colnames=True, max_len=2)

    rules = association_rules(frequent_items, metric='lift', min_threshold=1)
    result = rules.sort_values("confidence", ascending=False)  
    result = result[result["confidence"] >= min_con]
    result = result[result["lift"] > 1]

    return result

In [ ]:
ar = arm(data_b, min_sup= , min_con= )

In [ ]:
name=["antecedents","consequents","confidence"]
ar=ar[name]

In [ ]:
def list_to_matrix(data):
    """
    Function:Convert association rule data to matrix
    """
    G=nx.DiGraph(data)
    A=nx.to_numpy_array(G)
    return G,A,np.array(list(G.nodes()))

In [ ]:
G,A,n = list_to_matrix(ar)

In [ ]:
def node_tran(data,node):
    """
    Function: node transformation (nodes of association rules are frozen and need to be unfrozen)
    Input: nodes output when association rules are transformed into matrices, the earliest manually input nodes
    Output: nodes after unfreezing
    """
    new_node = []
    for no in data:
        for nd in node:
            if no == frozenset({nd}):
                new_node.append(nd)
    return np.array(new_node)

In [ ]:
n_new = node_tran(n,node)

In [ ]:
def in_degree(G):

    nodes = G.nodes()
    edges = G.edges()
    nodes_list = [key for key in nodes.keys()]
    edges_list = [key for key in edges.keys()]
    LN = len(nodes_list)
    LG = len(edges_list)
    k_in = []
    for i in range(LN):
        k_in.append(0) 
        for j in range(LG):
            if nodes_list[i] == edges_list[j][1]:
                k_in[i] += 1
    d1 = zip(nodes_list,k_in) 
    d11 = dict(d1)
    return d11

In [ ]:
def out_degree(G):

    nodes = G.nodes()
    edges = G.edges()
    nodes_list = [key for key in nodes.keys()]
    edges_list = [key for key in edges.keys()]
    LN = len(nodes_list)
    LG = len(edges_list)
    k_out = []
    for i in range(LN):
        k_out.append(0) 
        for j in range(LG):
            if nodes_list[i] == edges_list[j][0]:
                k_out[i] += 1
    d2 = zip(nodes_list,k_out) 
    d22 = dict(d2)
    return d22

In [ ]:
def in_strong_degree(G):

    nodes = G.nodes()
    edges = G.edges()
    w=nx.get_edge_attributes(G,'weight')
    nodes_list = [key for key in nodes.keys()]
    edges_list = [key for key in edges.keys()]
    dict_items=w.items()
    w_list = list(dict_items)

    LN = len(nodes_list)
    LG = len(edges_list)
    k_in_strong = []
    for i in range(LN):
        k_in_strong.append(0) 
        for j in range(LG):
            if nodes_list[i] == edges_list[j][1]:
                k_in_strong[i] += w_list[j][1]
    d3 = zip(nodes_list,k_in_strong) 
    d33 = dict(d3)
    return d33

In [ ]:
def out_strong_degree(G):

    nodes = G.nodes()
    edges = G.edges()
    w=nx.get_edge_attributes(G,'weight')
    nodes_list = [key for key in nodes.keys()]
    edges_list = [key for key in edges.keys()]
    dict_items=w.items()
    w_list = list(dict_items)
    LN = len(nodes_list)
    LG = len(edges_list)
    k_out_strong = []
    for i in range(LN):
        k_out_strong.append(0) 
        for j in range(LG):
            if nodes_list[i] == edges_list[j][0]:
                k_out_strong[i] += w_list[j][1]
    d4 = zip(nodes_list,k_out_strong) 
    d44 = dict(d4)
    return d44

In [ ]:
def efficiency(G):
    a=0
    for v in G.nodes():
        sp=nx.single_source_shortest_path_length(G,source=v)
        tn=[value for value in sp.values()]
        for i in range(len(tn)):
            if tn[i] != 0:
                a += (1/tn[i])
    n = len(G.nodes())
    if n>1:
        eff = a/(n*(n-1)) 
    else:
        eff=0
    return eff

In [ ]:
def graph_index(G, node):
    """
    Various metrics for complex networks
    Input: network,node
    """

    ne = pd.DataFrame([G.number_of_edges()], columns=[''])
    nn = pd.DataFrame([G.number_of_nodes()], columns=[''])
    nod = pd.DataFrame(node, columns=[''])
    de = []
    for i in np.array(G.degree):
        de.append(i[1])
    de = pd.DataFrame(de, columns=[""])
    in_de = pd.DataFrame(list(in_degree(G).values()), columns=[""])
    out_de = pd.DataFrame(list(out_degree(G).values()), columns=[""])
    in_s_de = pd.DataFrame(list(in_strong_degree(G).values()), columns=[""])
    out_s_de = pd.DataFrame(list(out_strong_degree(G).values()), columns=[""])
    cl = pd.DataFrame(list(nx.clustering(G).values()), columns=[""])
    acl = pd.DataFrame([nx.average_clustering(G)], columns=[""])
    cc = pd.DataFrame(nx.closeness_centrality(G).values(), columns=[''])
    bc = pd.DataFrame(nx.betweenness_centrality(G).values(), columns=[''])
    tran = pd.DataFrame([nx.transitivity(G)], columns=[''])
    dens = pd.DataFrame([nx.density(G)], columns=[''])
    eff = pd.DataFrame([efficiency(G)], columns=[''])

    frames = [nn, ne, nod, de, in_de, out_de, in_s_de, out_s_de, cl, cc, bc, acl, tran, dens, eff]  
    data_index = pd.concat(frames, axis=1)

    return data_index

In [ ]:
index=graph_index(G,n_new)

In [ ]:
def new_cn_m(data, node_o, node_all):
    """
    Constructing a new complex network matrix
    Inputs: original matrix, nodes of original matrix, all nodes
    """

    num = len(node_all)
    new_m = np.zeros((num, num))
    for i in range(len(node_o)):
        node1 = node_o[i]  
        for j in range(num):
            if node1 == node_all[j]:
                m = j  

        for k in range(len(node_o)):
            node2 = node_o[k]  
            for l in range(num):
                if node2 == node_all[l]:
                    n = l  
            new_m[m, n] = data[i, k]
    return new_m

In [ ]:
cm = new_cn_m(data=A,node_o=n_new,node_all=node)

In [ ]:
def cn_different(cn1, cn2, node):
    """
    Degree of dissimilarity of the two networks (including node dissimilarity and edge dissimilarity)
    Input: new complex network matrix of two networks, so nodes
    Output: node dissimilarity, edge dissimilarity, node case, edge case
    """


    a_n = []  
    b_n = []  
    c_n = []  

    for i in range(len(cn1)):
        n1 = sum(cn1[i])
        n2 = sum(cn2[i])
        if n1 > 0 and n2 > 0:
            a_n.append(node[i])
        if n1 > 0 and n2 == 0:
            b_n.append(node[i])
        if n1 == 0 and n2 > 0:
            c_n.append(node[i])

    n_diff = ((len(a_n) + len(b_n) + len(c_n)) * 2 / (2 * len(a_n) + len(b_n) + len(c_n))) - 1


    a_e = []  
    b_e = []  
    c_e = []  

    for i in range(len(cn1)):
        for j in range(len(cn1)):
            if cn1[i, j] > 0 and cn2[i, j] > 0:
                a_e.append([node[i], node[j]])
            if cn1[i, j] > 0 and cn2[i, j] == 0:
                b_e.append([node[i], node[j]])
            if cn1[i, j] == 0 and cn2[i, j] > 0:
                c_e.append([node[i], node[j]])

    e_diff = ((len(a_e) + len(b_e) + len(c_e)) * 2 / (2 * len(a_e) + len(b_e) + len(c_e))) - 1

    return n_diff, e_diff, [a_n, b_n, c_n], [a_e, b_e, c_e]


In [ ]:
node_diff,edge_diff,nnode,eedge=cn_different(cn1=cm,cn2=cm_c,node=node)

In [ ]:
def influence_matrix(data, cn, node):
    """
    Find the direct influence matrix (add diagonal to complex network matrix) (diagonal value is support)
    Inputs: raw data, raw complex network matrix, corresponding nodes of the raw complex network
    """
    im = cn.copy()

    for i in range(len(im)):
        nod = node[i]
        sup = 0
        for j in range(len(data)):
            for k in range(len(data.iloc[0])):
                if data.iloc[j][k] == nod:
                    sup += 1
        im[i, i] = (sup / len(data))

    return im

In [ ]:
im= influence_matrix(data, cn=A, node=n_new)

In [ ]:
def Normalized_influence_matrix(data):
    """
    Normalise the direct impact matrix with the input data as the direct impact matrix
    """

    a = []
    for i in data:
        a.append(np.sum(i))


    para = np.sum(a)

    nor_ma = data / para

    return nor_ma

In [ ]:
nm = Normalized_influence_matrix(im)

In [ ]:
def com_influence_matrix(data):
    """
    Transformation of the normative matrix into an integrated impact matrix
    """

    E = np.eye(len(data))

    T1 = np.linalg.inv(E - data)
    T = np.dot(data, T1)

    return T

In [ ]:
cim = com_influence_matrix(nm)

In [ ]:
ef WINGS_index(data):
    """
    Calculation of influence, influenced, centrality, cause and weight of each factor
    """

    D = []
    for i in data:
        D.append(np.sum(i))


    C = []
    for i in range(len(data)):
        cc = 0
        for j in range(len(data)):
            cc += data[j][i]
        C.append(cc)


    M = []
    for i in range(len(data)):
        M.append(D[i] + C[i])


    R = []
    for i in range(len(data)):
        R.append(D[i] - C[i])


    w = np.zeros(len(data))
    ww = 0
    for i in range(len(data)):
        ww1 = (M[i] ** 2 + R[i] ** 2) ** (1 / 2)
        ww += ww1
        w[i] = ww1
    w = w / ww

    return D, C, M, R, list(w)


In [ ]:
D, C, M, R, W = WINGS_index(cim)

In [ ]:
def lamda(data):
    """
    Intercept
    Input data: integrated impact matrix
    """

    x = np.mean(data)

    summ = 0
    num = len(data)
    for i in range(num):
        for j in range(num):
            summ += (data[i, j] - x) ** 2
    sigma = (summ / (num ** 2)) ** (1 / 2)

    return x + sigma

In [ ]:
def Relationship_Matrix(data, lamda):
    """
    Relationship matrix A

    Input data Combined impact matrix, intercept
    """

    A = np.zeros((len(data), len(data)))

    for i in range(len(data)):
        for j in range(len(data)):
            if data[i, j] > lamda:
                A[i, j] = 1

    return A

In [ ]:
RM = Relationship_Matrix(data=cim, lamda=lamda(cim))

In [ ]:
def Boolean_operation(A, B):
    """
    Boolean multiplication
    """
    bol = np.zeros((len(A), len(B[0])))

    for i in range(len(A)):
        for j in range(len(B[0])):
            num = 0
            for m in range(len(B)):
                if A[i, m] >= 1 and B[m, j] >= 1:
                    num += 1
            if num > 0:
                bol[i, j] = 1

    return bol

In [ ]:
def reachable_matrix(data):
    """
    Finding the reach matrix
    Input data: Relationship matrix
    """

    E = np.eye(len(data))
    B = data + E

    B1 = B
    B2 = np.zeros((len(data), len(data)))
    h = np.ones(len(data))
    while np.linalg.norm((B1 - B2)) != 0:
        B2 = B1
        B1 = Boolean_operation(B1, B)

    R = B1
    return R

In [ ]:
Reach = reachable_matrix(RM)

In [ ]:
def tarjan(graph):
    """
    Function: Finding strongly connected components
    Input data: reachable matrix
    Output: index
    """
    n = len(graph)
    index_counter = [0]
    index = [-1] * n
    lowlink = [-1] * n
    onStack = [False] * n
    stack = []

    result = []

    def strongconnect(v):
        index[v] = index_counter[0]
        lowlink[v] = index_counter[0]
        index_counter[0] += 1
        stack.append(v)
        onStack[v] = True

        for w in range(n):
            if graph[v][w] == 1:
                if index[w] == -1:
                    strongconnect(w)
                    lowlink[v] = min(lowlink[v], lowlink[w])
                elif onStack[w]:
                    lowlink[v] = min(lowlink[v], index[w])

        if lowlink[v] == index[v]:
            scc = []
            while True:
                w = stack.pop()
                onStack[w] = False
                scc.append(w)
                if w == v:
                    break
            result.append(scc)

    for v in range(n):
        if index[v] == -1:
            strongconnect(v)

    return result

In [ ]:
strcon = tarjan(Reach)

In [ ]:
st_con=[strcon[]]

In [ ]:
def shrink(R, st_con, lst):
    """
    Find the reduced point matrix , reduce the strongly connected points to a single point.
    Input data: reachable matrix, strongly connected components), name of each row
    """
    n1 = len(st_con)
    n2 = 0
    for i in st_con:
        n2 += len(i)
    n3 = len(R)

    n = n3 - n2 + n1

    S = np.zeros((n, n))  

    ind = []
    for i in range(n3):
        num = 0
        for j in st_con:
            if i in j:
                num += 1
        if num == 0:
            ind.append(i)


    for i in range(n):

        if i < len(ind):  
            for j in range(n):
                if j < len(ind):
                    S[i, j] = R[ind[i], ind[j]]
                else:
                    ss = 0
                    for m in st_con[j - len(ind)]:
                        ss += R[ind[i], m]
                    if ss == 0:
                        S[i, j] = 0
                    else:
                        S[i, j] = 1

        else:  
            for j in range(n):
                if j < len(ind):
                    ss = 0
                    for m in st_con[i - len(ind)]:
                        ss += R[m, ind[j]]

                    if ss == 0:
                        S[i, j] = 0
                    else:
                        S[i, j] = 1
                else:

                    ss = 0
                    for m in st_con[i - len(ind)]:
                        for k in st_con[j - len(ind)]:
                            ss += R[m, k]
                    if ss == 0:
                        S[i, j] = 0
                    else:
                        S[i, j] = 1




    labels = []
    for i in ind:
        labels.append(lst[i])
    for i in range(n1):
        labels.append("Shrink" + str(i + 1))
    return S, labels

In [ ]:
SN, labels = shrink(Reach, st_con, n_new)

In [ ]:
def s_edge(SN):
    """
 Reduced-edge matrix, resulting in a skeleton matrix
    """
    E = np.eye(len(SN))
    B = Boolean_operation((SN - E), (SN - E))
    SE = SN - B - E

    return SE

In [ ]:
SE = s_edge(SN)

In [ ]:
def show(data, lst):
    """
    Visualisation of who is connected to whom
    Input data, skeleton matrix, labels
    """
    n = len(data)
    sh = []
    for i in range(n):
        ss = ["" + lst[i]]
        for j in range(n):
            if data[i, j] == 1:
                ss.append(lst[j])
        if len(ss) == 1:
            ss.append(None)
        sh.append(ss)
    return sh

In [ ]:
sh=show(SE, labels)

In [ ]:
def RS_A(data, lis):
    """
Reachable sets
Input data: skeleton matrix, name of each column of the matrix
    """
    RS = []
    for i in range(len(data)):
        ss = []
        for j in range(len(data)):
            if data[i, j] == 1:
                ss.append(lis[j])
        RS.append(ss)
    return RS

In [ ]:
def QS_A(data, lis):
    """
    Prior set
    Input data: skeleton matrix, name of each column of the matrix
    """
    QS = []
    for i in range(len(data)):
        ss = []
        for j in range(len(data)):
            if data[j, i] == 1:
                ss.append(lis[j])
        QS.append(ss)
    return QS

In [ ]:
def TS_A(RS, QS):
    """
Intersection of reachable and prior sets
    """
    TS = []
    for i in range(len(RS)):
        ss = []
        for j in RS[i]:
            if j in QS[i]:
                ss.append(j)
        TS.append(ss)
    return TS

In [ ]:
def up_and_down_extract(data, lis):
    """
    up and down type extraction.
    Input data: reachable matrix, name of each column of the matrix
    """
    up = []
    down = []
    R = data.copy()

    RS = RS_A(R, lis)
    QS = QS_A(R, lis)
    TS1 = TS_A(RS, QS)
    TS2 = TS_A(RS, QS)
    num = np.sum(R)

    while num != 0:


        uup = []
        for i in range(len(RS)):
            if len(RS[i]) != 0:
                if RS[i] == TS1[i]:
                    for j in RS[i]:
                        uup.append(j)  

        uup = list(set(uup))
        up.append(uup)


        RS_n = []
        for i in RS:
            ss = []
            for j in i:
                if j not in uup:
                    ss.append(j)
            RS_n.append(ss)
        RS = RS_n

        TS1_n = []
        for i in TS1:
            ss = []
            for j in i:
                if j not in uup:
                    ss.append(j)
            TS1_n.append(ss)
        TS1 = TS1_n


        ddon = []
        for i in range(len(QS)):
            if len(QS[i]) != 0:
                if QS[i] == TS2[i]:
                    for j in QS[i]:
                        ddon.append(j)  

        ddon = list(set(ddon))
        down.append(ddon)

        QS_n = []
        for i in QS:
            ss = []
            for j in i:
                if j not in ddon:
                    ss.append(j)
            QS_n.append(ss)
        QS = QS_n

        TS2_n = []
        for i in TS2:
            ss = []
            for j in i:
                if j not in ddon:
                    ss.append(j)
            TS2_n.append(ss)
        TS2 = TS2_n


        num = 0
        for i in RS:
            num += len(i)
        for i in QS:
            num += len(i)


    up

    down_n = []
    for i in range(len(down)):
        down_n.append(down[-(i + 1)])
    down = down_n

    return up, down

In [ ]:
up, down = up_and_down_extract(Reach,n_new)